# <div style="text-align: right;font-family:Times New Roman">Приложение Б-5</div>
# <div style="text-align: center;font-family:Times New Roman">Тестирование модели на других территориях</div>

# Testing model on other territories
# Тестирование модели на других территориях
---
In this notebook the tree species model is tested on other territories data. It was tested on data from Noyabrsk forestry of Yamal-Nenets AO and three forestries of Krasnoyarsk region 

В данном блокноте модель преобладающих пород тестируется на даннных других территорий. Она тестировалась на данных Ноябрьского лесничества ЯНАО и трех лесничеств Красноярского края

### Importing dependencies
### Импорт необходимых библиотек

In [9]:
import os
import random
import re
import numpy as np
import matplotlib
from glob import glob
import rasterio as rio
import matplotlib.pyplot as plt
from matplotlib import patches as mpatches, colors
import pandas as pd
import pickle
from osgeo import ogr, osr

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils

In [10]:
tile_size = 128

In [57]:
#getting x data / список данных-предикторов
#x data is created using Sentinel-2 preprocessing notebook 
#даннные-предикторы созданы при помощи блокнота 'Предобработка Sentinel-2'
path = r'F:\Work\CorrProizv\tests\krasnoyarsk\irb\\'
files = glob(path+'B*.tif')
files.append(path + 'DEM.tif')
files.append(path + 'Landcover.tif')
files.append(path + 'Hansen_mosaic.tif')
files

['F:\\Work\\CorrProizv\\tests\\krasnoyarsk\\irb\\B1.tif',
 'F:\\Work\\CorrProizv\\tests\\krasnoyarsk\\irb\\B11.tif',
 'F:\\Work\\CorrProizv\\tests\\krasnoyarsk\\irb\\B12.tif',
 'F:\\Work\\CorrProizv\\tests\\krasnoyarsk\\irb\\B2.tif',
 'F:\\Work\\CorrProizv\\tests\\krasnoyarsk\\irb\\B3.tif',
 'F:\\Work\\CorrProizv\\tests\\krasnoyarsk\\irb\\B4.tif',
 'F:\\Work\\CorrProizv\\tests\\krasnoyarsk\\irb\\B5.tif',
 'F:\\Work\\CorrProizv\\tests\\krasnoyarsk\\irb\\B6.tif',
 'F:\\Work\\CorrProizv\\tests\\krasnoyarsk\\irb\\B7.tif',
 'F:\\Work\\CorrProizv\\tests\\krasnoyarsk\\irb\\B8.tif',
 'F:\\Work\\CorrProizv\\tests\\krasnoyarsk\\irb\\B8A.tif',
 'F:\\Work\\CorrProizv\\tests\\krasnoyarsk\\irb\\B9.tif',
 'F:\\Work\\CorrProizv\\tests\\krasnoyarsk\\irb\\\\DEM.tif',
 'F:\\Work\\CorrProizv\\tests\\krasnoyarsk\\irb\\\\Landcover.tif',
 'F:\\Work\\CorrProizv\\tests\\krasnoyarsk\\irb\\\\Hansen_mosaic.tif']

In [58]:
y = path + 'y.tif'
y

'F:\\Work\\CorrProizv\\tests\\krasnoyarsk\\irb\\\\y.tif'

In [76]:
#getting shape, proj and transform of input image / получение размеров, проекции и трансформации входных изображений
with rio.open(files[0]) as f:
    shp_in = f.shape
    transform = f.transform
    crs = f.crs
    dtype = f.dtypes[0]
#getting shape that is needed to divide into tiles without remainder 
#вычисление размераа, который нужен для разделения растра на тайлы без остатка
shp_work = []
for d in shp_in:
    while not d % tile_size == 0:
        d += 1
    shp_work.append(d)

In [60]:
#getting how much rows and cols we need to add to input shape 
#вычисление количества столбцов и строк, которые надо добавить к входному изображению
shp_pad = []
for i in range(len(shp_work)):
    a = shp_work[i]
    b = shp_in[i]
    shp_pad.append(a - b)

In [61]:
#creating numpy array / создание массива numpy
x_train_conv = np.empty((shp_work[0], shp_work[1], 15), dtype = 'uint16')

In [62]:
#reading all x files / чтение всех данных-предикторов
for i in range(len(files)):
    with rio.open(files[i]) as bnd:
        container = x_train_conv[:,:,i:i+1]
        container[:] = np.transpose(np.pad(bnd.read(), ((0,0),(0,shp_pad[0]),(0,shp_pad[1])), mode='constant', constant_values=0), axes=[1, 2, 0]).astype('uint16')
        print(files[i])

F:\Work\CorrProizv\tests\krasnoyarsk\irb\B1.tif
F:\Work\CorrProizv\tests\krasnoyarsk\irb\B11.tif
F:\Work\CorrProizv\tests\krasnoyarsk\irb\B12.tif
F:\Work\CorrProizv\tests\krasnoyarsk\irb\B2.tif
F:\Work\CorrProizv\tests\krasnoyarsk\irb\B3.tif
F:\Work\CorrProizv\tests\krasnoyarsk\irb\B4.tif
F:\Work\CorrProizv\tests\krasnoyarsk\irb\B5.tif
F:\Work\CorrProizv\tests\krasnoyarsk\irb\B6.tif
F:\Work\CorrProizv\tests\krasnoyarsk\irb\B7.tif
F:\Work\CorrProizv\tests\krasnoyarsk\irb\B8.tif
F:\Work\CorrProizv\tests\krasnoyarsk\irb\B8A.tif
F:\Work\CorrProizv\tests\krasnoyarsk\irb\B9.tif
F:\Work\CorrProizv\tests\krasnoyarsk\irb\\DEM.tif
F:\Work\CorrProizv\tests\krasnoyarsk\irb\\Landcover.tif
F:\Work\CorrProizv\tests\krasnoyarsk\irb\\Hansen_mosaic.tif


In [63]:
#reading y file / чтение обучающих данных
with rio.open(y) as bnd:
    y_train_conv = bnd.read().astype('uint16')
y_train_conv = np.pad(y_train_conv, ((0,0),(0,shp_pad[0]),(0,shp_pad[1])), mode='constant', constant_values=0)
y_train_conv = np.transpose(y_train_conv, axes=[1, 2, 0])

In [64]:
#generating tiles / генерация тайлов
tiles = []
x1 = 0
y1 = 0
x2 = tile_size
y2 = tile_size
for i in range (0,int(shp_work[0] / tile_size),1):
    for i in range(0,int(shp_work[1] / tile_size),1):
        tiles.append((x1,y1,x2,y2))
        y1+=tile_size
        y2+=tile_size
    y1=0
    y2=tile_size
    x1+=tile_size
    x2+=tile_size

In [65]:
#generating x tiles / генерация тайлов данных-предикторов
x = np.empty((len(tiles), tile_size, tile_size, 15), dtype = 'uint16')

In [66]:
for i in range(len(tiles)):
    c = tiles[i]
    container = x[i:i+1,:,:,:]
    container[:] = x_train_conv[c[0]:c[2], c[1]:c[3],:]

In [67]:
#generating y tiles / генерация тайлов обучающих данных
y = np.empty((len(tiles), tile_size, tile_size, 1), dtype = 'uint16')

In [68]:
for i in range(len(tiles)):
    c = tiles[i]
    container = y[i:i+1,:,:,:]
    container[:] = y_train_conv[c[0]:c[2], c[1]:c[3],:]

In [69]:
y = utils.to_categorical(y, num_classes=14, dtype = 'float16')

In [25]:
#loading model / загрузка модели
model = tf.keras.models.load_model(r'E:\Work\CorrProizv\deeplab_species1_1.hdf5')

In [70]:
class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, X_ds, y_ds, batch_size, *args, **kwargs):
        self.batch_size = batch_size
        self.X_ds = X_ds
        self.y_ds = y_ds

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.X_ds) / self.batch_size))

    def __getitem__(self, index):
        # returns one batch
        X = self.X_ds[index*self.batch_size:(index+1)*self.batch_size].astype('float32')
        y = self.y_ds[index*self.batch_size:(index+1)*self.batch_size].astype('float32')
        for i in range(len(X)):
            X[i] = X[i] * np.broadcast_to((1-y[i][:,:,0:1]), X[i].shape)
        return X, y

batch_size = 16
test_generator = CustomDataGen(X_ds = x, y_ds = y, batch_size = batch_size)

In [71]:
#testing / тестирование
results = model.evaluate(test_generator)

487/487 [==============================] - 35s 70ms/step - loss: 1.2071 - accuracy: 0.6712 - precision: 0.8147 - recall: 0.6356 - MAE: 0.0475 - auc: 0.9528


In [72]:
#creating numpy array for prediction results mapping / создание массива для картографирования результатов моделирования
y_pred = np.zeros(shp_work, dtype = 'int16')

In [73]:
#predictng / моделирование
for i in range(len(tiles)):
    prediction = model.predict(x[i:i+1])
    prediction = np.squeeze(np.argmax(prediction, axis=-1))
    t = tiles[i]
    y_pred[t[0]:t[2],t[1]:t[3]] = prediction

In [74]:
y_pred = y_pred[:shp_in[0], :shp_in[1]]

In [75]:
#saving map into raster file / сохранение карты в растровый файл
with rio.open(
    path + 'y_pred_1.tif',
    'w',
    driver='GTiff',
    height=y_pred.shape[0],
    width=y_pred.shape[1],
    count=1,
    dtype=y_pred.dtype,
    compress = 'deflate',
    PREDICTOR = 1,
    ZLEVEL=9,
    crs=crs,
    transform=transform,
    nodata = 0
) as outfile:
    outfile.write(y_pred, 1)

#### Tools for tile generation, semantic segmentation and mapping of the modeling results from this notebook are now avaliable in a Python library named [Remote Sensing Processor](https://github.com/simonreise/remote-sensing-processor)
#### Инструменты для генерации тайлов, семантической сегментации и картирования результатов моделирования, использованные в этом блокноте доступны в библиотеке Python [Remote Sensing Processor](https://github.com/simonreise/remote-sensing-processor)